<a href="https://colab.research.google.com/github/imabari/ImabariScraping/blob/master/Invisible_Intersection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 都道府県コード

https://www.npa.go.jp/publications/statistics/koutsuu/opendata/2021/codebook_2021.pdf

In [1]:
pref = 45 #@param {type:"integer"}


# 日本測地系2011（JGD2011）

https://lemulus.me/column/epsg-list-gis#i

In [2]:
epsg = 6677 #@param {type:"integer"}


In [3]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 53.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 64.8 MB/s eta 0:00:00


# プログラム

In [4]:
import pandas as pd
import geopandas as gpd

In [5]:
from shapely.geometry import Point

In [59]:
from tqdm.notebook import tqdm

In [6]:
pd.options.plotting.backend = "plotly"
pd.set_option("display.max_columns", None)

In [7]:
df0 = pd.read_csv("https://www.npa.go.jp/publications/statistics/koutsuu/opendata/2021/honhyo_2021.csv", encoding="cp932")

In [8]:
# 緯度経度を度分秒から10進数表記に変換

def dms2deg(df_tmp, col):

    df = df_tmp.copy()
    
    df["d"], df["t"] = df[col].divmod(10000000)
    df["m"], df["s"] = df["t"].divmod(100000)

    result = df["d"] + (df["m"] / 60.0) + (df["s"] / 1000.0 / 3600.0 )

    return result

In [9]:
df0["lat"] = dms2deg(df0, "地点　緯度（北緯）")
df0["lon"] = dms2deg(df0, "地点　経度（東経）")

In [10]:
df0

,資料区分,都道府県コード,警察署等コード,本票番号,事故内容,死者数,負傷者数,路線コード,上下線,地点コード,市区町村コード,発生日時 年,発生日時 月,発生日時 日,発生日時 時,発生日時 分,昼夜,天候,地形,路面状態,道路形状,環状交差点の直径,信号機,一時停止規制 標識（当事者A）,一時停止規制 表示（当事者A）,一時停止規制 標識（当事者B）,一時停止規制 表示（当事者B）,車道幅員,道路線形,衝突地点,ゾーン規制,中央分離帯施設等,歩車道区分,事故類型,年齢（当事者A）,年齢（当事者B）,当事者種別（当事者A）,当事者種別（当事者B）,用途別（当事者A）,用途別（当事者B）,車両形状（当事者A）,車両形状（当事者B）,速度規制（指定のみ）（当事者A）,速度規制（指定のみ）（当事者B）,車両の衝突部位（当事者A）,車両の衝突部位（当事者B）,車両の損壊程度（当事者A）,車両の損壊程度（当事者B）,エアバッグの装備（当事者A）,エアバッグの装備（当事者B）,サイドエアバッグの装備（当事者A）,サイドエアバッグの装備（当事者B）,人身損傷程度（当事者A）,人身損傷程度（当事者B）,地点 緯度（北緯）,地点 経度（東経）,曜日(発生年月日),祝日(発生年月日),lat,lon
0,1,10,59,1,2,0,1,40020,2,0,224,2020,12,28,20,25,22,2,3,1,12,0,7,0,23,0,23,4,9,1,70,1,4,21,65,35,3,13,31,31,1,11,10,10,50,40,3,3,2,2,2,2,4,2,424857124,1413708842,2,3,42.815868,141.619123
1,1,10,59,2,2,0,2,40040,2,0,222,2020,12,26,14,30,12,5,3,4,14,0,7,0,23,0,23,4,9,1,70,1,4,21,25,45,11,3,1,31,11,1,4,4,10,30,3,2,2,2,2,2,4,2,431343622,1414949274,7,3,43.228784,141.830354
2,1,10,59,3,2,0,4,40030,1,0,222,2020,12,2,16,55,21,2,3,4,14,0,7,0,23,0,23,4,9,1,70,1,4,21,55,25,3,13,31,31,1,11,4,4,80,60,3,3,2,2,2,2,2,4,431438483,1414949168,4,3,43.244023,141.830324
3,1,10,59,4,2,0,1,50020,2,0,213,2020,12,7,7,10,11,3,3,3,14,0,7,0,23,0,23,4,9,1,70,1,4,21,25,1,11,13,1,31,11,11,10,10,78,56,3,2,2,2,2,2,4,4,424009246,1414739252,2,3,42.669235,141.794237
4,1,10,101,1,2,0,1,39990,0,0,101,2020,12,26,18,40,22,1,1,3,14,0,7,0,23,0,23,4,9,1,70,5,2,1,65,1,3,61,1,0,1,0,4,0,20,0,4,0,2,0,2,0,4,2,430320435,1412105782,7,3,43.055676,141.351606
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305191,1,97,114,166,2,0,4,33410,0,0,212,2021,12,10,21,33,22,1,3,1,7,0,1,0,23,0,23,4,6,1,70,4,2,21,65,65,4,4,31,31,1,1,10,10,10,10,2,2,1,1,2,2,2,2,261031432,1273909676,6,3,26.175398,127.652688
305192,1,97,114,167,2,0,1,33410,0,0,212,2021,12,15,11,12,12,1,3,1,1,0,7,1,21,9,22,14,9,30,70,4,2,21,55,35,3,3,31,31,1,1,10,2,50,40,3,3,2,2,2,2,4,2,260918133,1274003673,4,3,26.155037,127.667687
305193,1,97,114,168,2,0,1,10070,0,0,212,2021,12,22,18,45,22,3,2,2,14,0,7,0,23,0,23,3,9,1,70,4,2,21,45,25,3,4,31,31,1,1,3,3,10,30,3,3,2,2,2,2,4,2,261022159,1274124558,4,3,26.172822,127.690155
305194,1,97,114,169,2,0,1,33410,0,0,212,2021,12,24,15,30,12,2,2,1,14,0,7,0,23,0,23,3,9,1,70,4,2,1,35,1,4,61,31,0,1,0,2,0,50,0,3,0,2,0,2,0,4,2,260955144,1273944244,6,3,26.165318,127.662290


In [11]:
df0.columns

Index(['資料区分', '都道府県コード', '警察署等コード', '本票番号', '事故内容', '死者数', '負傷者数', '路線コード',
       '上下線', '地点コード', '市区町村コード', '発生日時　　年', '発生日時　　月', '発生日時　　日', '発生日時　　時',
       '発生日時　　分', '昼夜', '天候', '地形', '路面状態', '道路形状', '環状交差点の直径', '信号機',
       '一時停止規制　標識（当事者A）', '一時停止規制　表示（当事者A）', '一時停止規制　標識（当事者B）',
       '一時停止規制　表示（当事者B）', '車道幅員', '道路線形', '衝突地点', 'ゾーン規制', '中央分離帯施設等', '歩車道区分',
       '事故類型', '年齢（当事者A）', '年齢（当事者B）', '当事者種別（当事者A）', '当事者種別（当事者B）',
       '用途別（当事者A）', '用途別（当事者B）', '車両形状（当事者A）', '車両形状（当事者B）',
       '速度規制（指定のみ）（当事者A）', '速度規制（指定のみ）（当事者B）', '車両の衝突部位（当事者A）',
       '車両の衝突部位（当事者B）', '車両の損壊程度（当事者A）', '車両の損壊程度（当事者B）', 'エアバッグの装備（当事者A）',
       'エアバッグの装備（当事者B）', 'サイドエアバッグの装備（当事者A）', 'サイドエアバッグの装備（当事者B）',
       '人身損傷程度（当事者A）', '人身損傷程度（当事者B）', '地点　緯度（北緯）', '地点　経度（東経）', '曜日(発生年月日)',
       '祝日(発生年月日)', 'lat', 'lon'],
      dtype='object')

In [12]:
df0["発生日時　　年"].value_counts()

2021    297088
2020      8069
2019        28
2018         7
2017         3
2016         1
Name: 発生日時　　年, dtype: int64

In [13]:
# 2021年のみ
df1 = df0[df0["発生日時　　年"] == 2021].copy()

In [14]:
df1.shape

(297088, 60)

In [15]:
# 月ごとの事故数を確認
df1["発生日時　　月"].value_counts()

10    27978
11    27871
7     25970
3     25683
4     25110
12    24568
6     24182
1     24048
2     23332
9     23033
8     22905
5     22408
Name: 発生日時　　月, dtype: int64

In [16]:
# 曜日別の事故数を確認
df1["曜日(発生年月日)"].value_counts()

6    49422
2    45249
5    44845
4    44746
3    44013
7    39340
1    29473
Name: 曜日(発生年月日), dtype: int64

In [17]:
# 昼夜別の事故数を確認
df1["昼夜"].value_counts()

12    194588
22     46159
21     22910
13     19812
11      9650
23      3969
Name: 昼夜, dtype: int64

In [18]:
# 道路の形ごとの事故数を確認
df1["道路線形"].value_counts()

9    249963
0     14638
8     11569
7      8109
3      4088
6      4075
5      1428
2      1262
1      1017
4       939
Name: 道路線形, dtype: int64

In [19]:
# 交差点の大きさごとの事故数を確認
df1["車道幅員"].value_counts()

3     70349
15    44087
14    32260
4     30088
11    23020
5     22226
2     19176
0     14638
18    12960
19     7727
1      7627
17     7102
6      5828
Name: 車道幅員, dtype: int64

In [20]:
df2 = df1[df1["車道幅員"].isin([11, 14, 15]) & (df1["都道府県コード"] == pref)]
df2

,資料区分,都道府県コード,警察署等コード,本票番号,事故内容,死者数,負傷者数,路線コード,上下線,地点コード,市区町村コード,発生日時 年,発生日時 月,発生日時 日,発生日時 時,発生日時 分,昼夜,天候,地形,路面状態,道路形状,環状交差点の直径,信号機,一時停止規制 標識（当事者A）,一時停止規制 表示（当事者A）,一時停止規制 標識（当事者B）,一時停止規制 表示（当事者B）,車道幅員,道路線形,衝突地点,ゾーン規制,中央分離帯施設等,歩車道区分,事故類型,年齢（当事者A）,年齢（当事者B）,当事者種別（当事者A）,当事者種別（当事者B）,用途別（当事者A）,用途別（当事者B）,車両形状（当事者A）,車両形状（当事者B）,速度規制（指定のみ）（当事者A）,速度規制（指定のみ）（当事者B）,車両の衝突部位（当事者A）,車両の衝突部位（当事者B）,車両の損壊程度（当事者A）,車両の損壊程度（当事者B）,エアバッグの装備（当事者A）,エアバッグの装備（当事者B）,サイドエアバッグの装備（当事者A）,サイドエアバッグの装備（当事者B）,人身損傷程度（当事者A）,人身損傷程度（当事者B）,地点 緯度（北緯）,地点 経度（東経）,曜日(発生年月日),祝日(発生年月日),lat,lon
8159,1,45,131,5,2,0,1,30270,0,0,104,2021,1,21,21,14,22,1,1,1,1,0,1,9,22,0,23,15,9,30,70,1,2,1,65,35,3,61,1,0,1,0,4,0,10,0,2,0,2,0,2,0,4,2,352643979,1393803504,5,3,35.445550,139.634307
8162,1,45,131,8,2,0,2,30000,0,0,104,2021,1,24,12,50,12,2,1,1,1,0,7,1,21,9,22,14,9,30,70,4,2,21,25,35,3,3,31,31,1,1,2,2,10,70,3,2,2,2,2,2,4,2,352634260,1393833592,1,3,35.442850,139.642664
8164,1,45,131,10,2,0,1,30260,0,0,104,2021,1,25,17,0,13,1,1,1,1,0,7,1,22,0,23,14,9,30,70,1,2,1,65,55,3,61,1,0,1,0,4,0,50,0,4,0,2,0,1,0,4,2,352633381,1393900130,2,3,35.442606,139.650036
8168,1,45,132,3,2,0,1,30000,0,0,104,2021,1,9,12,45,12,1,2,1,1,0,7,9,22,0,23,14,9,30,70,5,2,21,1,0,51,0,0,0,0,0,0,0,30,0,3,0,0,0,0,0,2,0,352527749,1393955249,7,3,35.424375,139.665347
8174,1,45,132,9,2,0,1,30000,0,0,104,2021,1,23,17,45,21,3,1,2,1,0,7,9,22,0,23,15,8,30,70,4,2,41,25,0,36,76,31,0,0,0,10,0,20,0,3,0,0,0,0,0,2,0,352538288,1393953598,7,3,35.427302,139.664888
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286255,1,45,295,272,2,0,1,30000,0,0,151,2021,12,25,11,35,12,1,1,1,1,0,1,9,22,9,22,15,8,30,70,4,2,21,45,45,35,51,31,0,0,0,3,0,30,20,4,4,0,0,0,0,4,2,353507086,1391939063,7,3,35.585302,139.327517
286258,1,45,295,275,2,0,1,30000,0,0,151,2021,11,26,11,20,12,1,1,1,1,0,7,9,22,0,23,11,9,30,70,4,2,41,55,0,1,76,1,0,1,0,3,0,0,0,4,0,2,0,2,0,4,0,353556641,1392019287,6,3,35.599067,139.338691
286259,1,45,295,276,2,0,1,30000,0,0,151,2021,12,28,12,25,12,1,1,1,1,0,7,9,22,1,22,14,9,30,70,5,3,21,75,1,4,51,31,0,1,0,10,0,80,10,3,3,2,0,2,0,4,2,353434168,1391853168,3,3,35.576158,139.314769
286260,1,45,295,277,2,0,2,160,1,303,151,2021,12,30,10,21,12,1,1,1,1,0,1,9,22,9,22,11,9,30,70,1,1,21,35,75,3,4,31,31,1,1,5,3,80,50,2,2,2,2,2,2,2,2,353605620,1392019796,5,3,35.601561,139.338832


In [21]:
geo_df = gpd.GeoDataFrame(df2, geometry = gpd.points_from_xy(df2.lon, df2.lat), crs=epsg)
geo_df

,資料区分,都道府県コード,警察署等コード,本票番号,事故内容,死者数,負傷者数,路線コード,上下線,地点コード,市区町村コード,発生日時 年,発生日時 月,発生日時 日,発生日時 時,発生日時 分,昼夜,天候,地形,路面状態,道路形状,環状交差点の直径,信号機,一時停止規制 標識（当事者A）,一時停止規制 表示（当事者A）,一時停止規制 標識（当事者B）,一時停止規制 表示（当事者B）,車道幅員,道路線形,衝突地点,ゾーン規制,中央分離帯施設等,歩車道区分,事故類型,年齢（当事者A）,年齢（当事者B）,当事者種別（当事者A）,当事者種別（当事者B）,用途別（当事者A）,用途別（当事者B）,車両形状（当事者A）,車両形状（当事者B）,速度規制（指定のみ）（当事者A）,速度規制（指定のみ）（当事者B）,車両の衝突部位（当事者A）,車両の衝突部位（当事者B）,車両の損壊程度（当事者A）,車両の損壊程度（当事者B）,エアバッグの装備（当事者A）,エアバッグの装備（当事者B）,サイドエアバッグの装備（当事者A）,サイドエアバッグの装備（当事者B）,人身損傷程度（当事者A）,人身損傷程度（当事者B）,地点 緯度（北緯）,地点 経度（東経）,曜日(発生年月日),祝日(発生年月日),lat,lon,geometry
8159,1,45,131,5,2,0,1,30270,0,0,104,2021,1,21,21,14,22,1,1,1,1,0,1,9,22,0,23,15,9,30,70,1,2,1,65,35,3,61,1,0,1,0,4,0,10,0,2,0,2,0,2,0,4,2,352643979,1393803504,5,3,35.445550,139.634307,POINT (139.634 35.446)
8162,1,45,131,8,2,0,2,30000,0,0,104,2021,1,24,12,50,12,2,1,1,1,0,7,1,21,9,22,14,9,30,70,4,2,21,25,35,3,3,31,31,1,1,2,2,10,70,3,2,2,2,2,2,4,2,352634260,1393833592,1,3,35.442850,139.642664,POINT (139.643 35.443)
8164,1,45,131,10,2,0,1,30260,0,0,104,2021,1,25,17,0,13,1,1,1,1,0,7,1,22,0,23,14,9,30,70,1,2,1,65,55,3,61,1,0,1,0,4,0,50,0,4,0,2,0,1,0,4,2,352633381,1393900130,2,3,35.442606,139.650036,POINT (139.650 35.443)
8168,1,45,132,3,2,0,1,30000,0,0,104,2021,1,9,12,45,12,1,2,1,1,0,7,9,22,0,23,14,9,30,70,5,2,21,1,0,51,0,0,0,0,0,0,0,30,0,3,0,0,0,0,0,2,0,352527749,1393955249,7,3,35.424375,139.665347,POINT (139.665 35.424)
8174,1,45,132,9,2,0,1,30000,0,0,104,2021,1,23,17,45,21,3,1,2,1,0,7,9,22,0,23,15,8,30,70,4,2,41,25,0,36,76,31,0,0,0,10,0,20,0,3,0,0,0,0,0,2,0,352538288,1393953598,7,3,35.427302,139.664888,POINT (139.665 35.427)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286255,1,45,295,272,2,0,1,30000,0,0,151,2021,12,25,11,35,12,1,1,1,1,0,1,9,22,9,22,15,8,30,70,4,2,21,45,45,35,51,31,0,0,0,3,0,30,20,4,4,0,0,0,0,4,2,353507086,1391939063,7,3,35.585302,139.327517,POINT (139.328 35.585)
286258,1,45,295,275,2,0,1,30000,0,0,151,2021,11,26,11,20,12,1,1,1,1,0,7,9,22,0,23,11,9,30,70,4,2,41,55,0,1,76,1,0,1,0,3,0,0,0,4,0,2,0,2,0,4,0,353556641,1392019287,6,3,35.599067,139.338691,POINT (139.339 35.599)
286259,1,45,295,276,2,0,1,30000,0,0,151,2021,12,28,12,25,12,1,1,1,1,0,7,9,22,1,22,14,9,30,70,5,3,21,75,1,4,51,31,0,1,0,10,0,80,10,3,3,2,0,2,0,4,2,353434168,1391853168,3,3,35.576158,139.314769,POINT (139.315 35.576)
286260,1,45,295,277,2,0,2,160,1,303,151,2021,12,30,10,21,12,1,1,1,1,0,1,9,22,9,22,11,9,30,70,1,1,21,35,75,3,4,31,31,1,1,5,3,80,50,2,2,2,2,2,2,2,2,353605620,1392019796,5,3,35.601561,139.338832,POINT (139.339 35.602)


In [73]:
df3 = df2.copy()

In [74]:
# 10mの範囲のみ

for i, r in tqdm(df3.iterrows()):
    
    point = Point(r.lon, r.lat)

    # 10mの範囲
    buffer = point.buffer(0.0001)

    dft = geo_df[geo_df.geometry.within(buffer)].copy().sort_index()

    df3.at[i, "len"] = len(dft)
    df3.at[i, "idx"] = "_".join(map(str, dft.index.values))

0it [00:00, ?it/s]

In [60]:
# 10mの範囲のポイントから10mの範囲
"""
for i, r in tqdm(df3.iterrows()):
    
    point = Point(r.lon, r.lat)

    # 10mの範囲
    buffer = point.buffer(0.0001)

    tmp = geo_df[geo_df.geometry.within(buffer)].copy()

    circles = tmp.geometry.buffer(0.0001).unary_union

    dft = geo_df[geo_df.geometry.within(circles)].copy().sort_index()

    df3.at[i, "len"] = len(dft)
    df3.at[i, "idx"] = "_".join(map(str, dft.index.values))
"""

0it [00:00, ?it/s]

In [75]:
df3

,資料区分,都道府県コード,警察署等コード,本票番号,事故内容,死者数,負傷者数,路線コード,上下線,地点コード,市区町村コード,発生日時 年,発生日時 月,発生日時 日,発生日時 時,発生日時 分,昼夜,天候,地形,路面状態,道路形状,環状交差点の直径,信号機,一時停止規制 標識（当事者A）,一時停止規制 表示（当事者A）,一時停止規制 標識（当事者B）,一時停止規制 表示（当事者B）,車道幅員,道路線形,衝突地点,ゾーン規制,中央分離帯施設等,歩車道区分,事故類型,年齢（当事者A）,年齢（当事者B）,当事者種別（当事者A）,当事者種別（当事者B）,用途別（当事者A）,用途別（当事者B）,車両形状（当事者A）,車両形状（当事者B）,速度規制（指定のみ）（当事者A）,速度規制（指定のみ）（当事者B）,車両の衝突部位（当事者A）,車両の衝突部位（当事者B）,車両の損壊程度（当事者A）,車両の損壊程度（当事者B）,エアバッグの装備（当事者A）,エアバッグの装備（当事者B）,サイドエアバッグの装備（当事者A）,サイドエアバッグの装備（当事者B）,人身損傷程度（当事者A）,人身損傷程度（当事者B）,地点 緯度（北緯）,地点 経度（東経）,曜日(発生年月日),祝日(発生年月日),lat,lon,len,idx
8159,1,45,131,5,2,0,1,30270,0,0,104,2021,1,21,21,14,22,1,1,1,1,0,1,9,22,0,23,15,9,30,70,1,2,1,65,35,3,61,1,0,1,0,4,0,10,0,2,0,2,0,2,0,4,2,352643979,1393803504,5,3,35.445550,139.634307,1.0,8159
8162,1,45,131,8,2,0,2,30000,0,0,104,2021,1,24,12,50,12,2,1,1,1,0,7,1,21,9,22,14,9,30,70,4,2,21,25,35,3,3,31,31,1,1,2,2,10,70,3,2,2,2,2,2,4,2,352634260,1393833592,1,3,35.442850,139.642664,1.0,8162
8164,1,45,131,10,2,0,1,30260,0,0,104,2021,1,25,17,0,13,1,1,1,1,0,7,1,22,0,23,14,9,30,70,1,2,1,65,55,3,61,1,0,1,0,4,0,50,0,4,0,2,0,1,0,4,2,352633381,1393900130,2,3,35.442606,139.650036,1.0,8164
8168,1,45,132,3,2,0,1,30000,0,0,104,2021,1,9,12,45,12,1,2,1,1,0,7,9,22,0,23,14,9,30,70,5,2,21,1,0,51,0,0,0,0,0,0,0,30,0,3,0,0,0,0,0,2,0,352527749,1393955249,7,3,35.424375,139.665347,1.0,8168
8174,1,45,132,9,2,0,1,30000,0,0,104,2021,1,23,17,45,21,3,1,2,1,0,7,9,22,0,23,15,8,30,70,4,2,41,25,0,36,76,31,0,0,0,10,0,20,0,3,0,0,0,0,0,2,0,352538288,1393953598,7,3,35.427302,139.664888,1.0,8174
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
286255,1,45,295,272,2,0,1,30000,0,0,151,2021,12,25,11,35,12,1,1,1,1,0,1,9,22,9,22,15,8,30,70,4,2,21,45,45,35,51,31,0,0,0,3,0,30,20,4,4,0,0,0,0,4,2,353507086,1391939063,7,3,35.585302,139.327517,1.0,286255
286258,1,45,295,275,2,0,1,30000,0,0,151,2021,11,26,11,20,12,1,1,1,1,0,7,9,22,0,23,11,9,30,70,4,2,41,55,0,1,76,1,0,1,0,3,0,0,0,4,0,2,0,2,0,4,0,353556641,1392019287,6,3,35.599067,139.338691,1.0,286258
286259,1,45,295,276,2,0,1,30000,0,0,151,2021,12,28,12,25,12,1,1,1,1,0,7,9,22,1,22,14,9,30,70,5,3,21,75,1,4,51,31,0,1,0,10,0,80,10,3,3,2,0,2,0,4,2,353434168,1391853168,3,3,35.576158,139.314769,1.0,286259
286260,1,45,295,277,2,0,2,160,1,303,151,2021,12,30,10,21,12,1,1,1,1,0,1,9,22,9,22,11,9,30,70,1,1,21,35,75,3,4,31,31,1,1,5,3,80,50,2,2,2,2,2,2,2,2,353605620,1392019796,5,3,35.601561,139.338832,1.0,286260


In [76]:
df3["len"].value_counts()

1.0    6460
2.0    1221
3.0     309
4.0      49
5.0      16
9.0       9
8.0       8
6.0       6
Name: len, dtype: int64

In [77]:
df4 = df3[df3["len"] > 5].copy()

In [78]:
df4

,資料区分,都道府県コード,警察署等コード,本票番号,事故内容,死者数,負傷者数,路線コード,上下線,地点コード,市区町村コード,発生日時 年,発生日時 月,発生日時 日,発生日時 時,発生日時 分,昼夜,天候,地形,路面状態,道路形状,環状交差点の直径,信号機,一時停止規制 標識（当事者A）,一時停止規制 表示（当事者A）,一時停止規制 標識（当事者B）,一時停止規制 表示（当事者B）,車道幅員,道路線形,衝突地点,ゾーン規制,中央分離帯施設等,歩車道区分,事故類型,年齢（当事者A）,年齢（当事者B）,当事者種別（当事者A）,当事者種別（当事者B）,用途別（当事者A）,用途別（当事者B）,車両形状（当事者A）,車両形状（当事者B）,速度規制（指定のみ）（当事者A）,速度規制（指定のみ）（当事者B）,車両の衝突部位（当事者A）,車両の衝突部位（当事者B）,車両の損壊程度（当事者A）,車両の損壊程度（当事者B）,エアバッグの装備（当事者A）,エアバッグの装備（当事者B）,サイドエアバッグの装備（当事者A）,サイドエアバッグの装備（当事者B）,人身損傷程度（当事者A）,人身損傷程度（当事者B）,地点 緯度（北緯）,地点 経度（東経）,曜日(発生年月日),祝日(発生年月日),lat,lon,len,idx
9575,1,45,275,46,2,0,1,30000,0,0,213,2021,1,19,11,53,12,1,2,1,1,0,4,9,22,9,22,15,9,30,70,5,3,21,75,55,4,4,31,31,1,1,2,2,50,47,3,3,2,2,2,2,4,2,352952796,1392605435,3,3,35.497999,139.434843,8.0,9575_33429_57759_57779_83671_255952_255973_285943
33429,1,45,275,154,2,0,1,30000,0,0,213,2021,2,22,14,55,12,1,2,1,1,0,7,9,22,9,22,11,9,30,70,5,4,21,25,45,3,3,31,1,1,1,10,10,50,80,3,3,2,2,2,2,4,2,352952856,1392605445,2,2,35.498016,139.434846,8.0,9575_33429_57759_57779_83671_255952_255973_285943
57473,1,45,268,131,2,0,1,30000,0,0,203,2021,3,5,10,1,12,2,1,1,1,0,1,9,22,9,22,14,9,30,70,4,3,21,35,55,3,3,31,31,1,1,3,3,80,50,1,1,2,2,2,2,4,2,352126505,1392142990,6,3,35.357363,139.361942,6.0,57473_130822_130842_130853_155471_179534
57759,1,45,275,209,2,0,1,30000,0,0,213,2021,2,17,7,5,11,1,1,1,1,0,7,9,22,0,23,15,9,30,70,5,4,21,45,0,3,0,31,0,1,0,2,0,50,0,3,0,1,0,2,0,2,0,352952856,1392605455,4,3,35.498016,139.434849,8.0,9575_33429_57759_57779_83671_255952_255973_285943
57779,1,45,275,229,2,0,1,30000,0,0,213,2021,3,23,13,20,12,1,1,1,1,0,4,9,22,9,22,15,9,30,70,5,3,21,1,1,4,51,31,0,1,0,2,0,10,20,3,3,2,0,2,0,4,2,352952846,1392605515,3,3,35.498013,139.434865,8.0,9575_33429_57759_57779_83671_255952_255973_285943
82687,1,45,191,177,2,0,2,30000,0,0,117,2021,4,22,14,27,12,2,2,1,1,0,7,1,21,9,22,14,9,30,70,4,2,21,25,65,3,35,31,31,1,0,10,3,10,40,3,3,2,0,2,0,4,2,353208536,1393032256,5,3,35.535704,139.508960,9.0,82687_82706_154854_179012_227315_254933_254947...
82706,1,45,191,196,2,0,1,30000,0,0,117,2021,4,24,7,0,12,1,2,1,1,0,7,1,21,9,22,15,9,30,70,4,2,21,65,35,3,3,31,31,1,1,10,3,50,80,3,3,2,2,2,2,4,2,353208577,1393032227,7,3,35.535716,139.508952,9.0,82687_82706_154854_179012_227315_254933_254947...
83671,1,45,275,268,2,0,1,30000,0,0,213,2021,4,7,8,26,12,1,2,1,1,0,4,9,22,9,22,15,9,30,70,5,3,21,55,45,17,3,1,31,11,1,2,2,80,50,3,3,2,2,2,2,4,2,352952886,1392605515,4,3,35.498024,139.434865,8.0,9575_33429_57759_57779_83671_255952_255973_285943
130822,1,45,268,332,2,0,2,30000,0,0,203,2021,6,13,14,40,12,2,1,1,1,0,1,9,22,9,22,14,9,30,70,4,4,21,45,35,3,4,31,31,1,1,1,3,10,40,2,2,2,2,2,2,4,4,352126546,1392142920,1,3,35.357374,139.361922,6.0,57473_130822_130842_130853_155471_179534
130842,1,45,268,352,2,0,1,30000,0,0,203,2021,6,22,10,56,12,2,1,1,1,0,1,9,22,9,22,15,9,30,70,4,3,21,45,65,3,3,31,31,1,1,3,3,50,80,3,3,2,2,2,2,4,2,352126515,1392142960,3,3,35.357365,139.361933,6.0,57473_130822_130842_130853_155471_179534


In [79]:
se = df4.groupby("idx").apply(len).rank(ascending=False).astype(int).apply(lambda x: f"intersection_{x}")
se.name = "intersection_name"

In [80]:
df5 = pd.merge(df4, se, on="idx")

In [81]:
df5

,資料区分,都道府県コード,警察署等コード,本票番号,事故内容,死者数,負傷者数,路線コード,上下線,地点コード,市区町村コード,発生日時 年,発生日時 月,発生日時 日,発生日時 時,発生日時 分,昼夜,天候,地形,路面状態,道路形状,環状交差点の直径,信号機,一時停止規制 標識（当事者A）,一時停止規制 表示（当事者A）,一時停止規制 標識（当事者B）,一時停止規制 表示（当事者B）,車道幅員,道路線形,衝突地点,ゾーン規制,中央分離帯施設等,歩車道区分,事故類型,年齢（当事者A）,年齢（当事者B）,当事者種別（当事者A）,当事者種別（当事者B）,用途別（当事者A）,用途別（当事者B）,車両形状（当事者A）,車両形状（当事者B）,速度規制（指定のみ）（当事者A）,速度規制（指定のみ）（当事者B）,車両の衝突部位（当事者A）,車両の衝突部位（当事者B）,車両の損壊程度（当事者A）,車両の損壊程度（当事者B）,エアバッグの装備（当事者A）,エアバッグの装備（当事者B）,サイドエアバッグの装備（当事者A）,サイドエアバッグの装備（当事者B）,人身損傷程度（当事者A）,人身損傷程度（当事者B）,地点 緯度（北緯）,地点 経度（東経）,曜日(発生年月日),祝日(発生年月日),lat,lon,len,idx,intersection_name
0,1,45,275,46,2,0,1,30000,0,0,213,2021,1,19,11,53,12,1,2,1,1,0,4,9,22,9,22,15,9,30,70,5,3,21,75,55,4,4,31,31,1,1,2,2,50,47,3,3,2,2,2,2,4,2,352952796,1392605435,3,3,35.497999,139.434843,8.0,9575_33429_57759_57779_83671_255952_255973_285943,intersection_2
1,1,45,275,154,2,0,1,30000,0,0,213,2021,2,22,14,55,12,1,2,1,1,0,7,9,22,9,22,11,9,30,70,5,4,21,25,45,3,3,31,1,1,1,10,10,50,80,3,3,2,2,2,2,4,2,352952856,1392605445,2,2,35.498016,139.434846,8.0,9575_33429_57759_57779_83671_255952_255973_285943,intersection_2
2,1,45,275,209,2,0,1,30000,0,0,213,2021,2,17,7,5,11,1,1,1,1,0,7,9,22,0,23,15,9,30,70,5,4,21,45,0,3,0,31,0,1,0,2,0,50,0,3,0,1,0,2,0,2,0,352952856,1392605455,4,3,35.498016,139.434849,8.0,9575_33429_57759_57779_83671_255952_255973_285943,intersection_2
3,1,45,275,229,2,0,1,30000,0,0,213,2021,3,23,13,20,12,1,1,1,1,0,4,9,22,9,22,15,9,30,70,5,3,21,1,1,4,51,31,0,1,0,2,0,10,20,3,3,2,0,2,0,4,2,352952846,1392605515,3,3,35.498013,139.434865,8.0,9575_33429_57759_57779_83671_255952_255973_285943,intersection_2
4,1,45,275,268,2,0,1,30000,0,0,213,2021,4,7,8,26,12,1,2,1,1,0,4,9,22,9,22,15,9,30,70,5,3,21,55,45,17,3,1,31,11,1,2,2,80,50,3,3,2,2,2,2,4,2,352952886,1392605515,4,3,35.498024,139.434865,8.0,9575_33429_57759_57779_83671_255952_255973_285943,intersection_2
5,1,45,275,803,2,0,1,30000,0,0,213,2021,11,4,18,45,22,1,2,1,1,0,7,2,21,9,22,11,9,30,70,5,3,21,55,55,3,51,31,0,1,0,10,0,10,40,3,3,2,0,2,0,4,2,352952986,1392605445,5,3,35.498052,139.434846,8.0,9575_33429_57759_57779_83671_255952_255973_285943,intersection_2
6,1,45,275,824,2,0,1,30000,0,0,213,2021,11,9,10,9,12,3,2,1,1,0,7,9,22,1,22,14,9,30,70,5,3,21,35,75,17,4,31,31,11,1,10,10,50,10,3,3,2,2,2,2,2,4,352952796,1392605445,3,3,35.497999,139.434846,8.0,9575_33429_57759_57779_83671_255952_255973_285943,intersection_2
7,1,45,275,969,2,0,1,30000,0,0,213,2021,12,25,8,40,12,1,1,1,1,0,7,9,22,9,21,11,9,30,70,5,3,21,55,45,4,4,31,31,1,1,10,10,10,60,2,1,1,2,1,2,4,2,352952876,1392605535,7,3,35.498021,139.434871,8.0,9575_33429_57759_57779_83671_255952_255973_285943,intersection_2
8,1,45,268,131,2,0,1,30000,0,0,203,2021,3,5,10,1,12,2,1,1,1,0,1,9,22,9,22,14,9,30,70,4,3,21,35,55,3,3,31,31,1,1,3,3,80,50,1,1,2,2,2,2,4,2,352126505,1392142990,6,3,35.357363,139.361942,6.0,57473_130822_130842_130853_155471_179534,intersection_3
9,1,45,268,332,2,0,2,30000,0,0,203,2021,6,13,14,40,12,2,1,1,1,0,1,9,22,9,22,14,9,30,70,4,4,21,45,35,3,4,31,31,1,1,1,3,10,40,2,2,2,2,2,2,4,4,352126546,1392142920,1,3,35.357374,139.361922,6.0,57473_130822_130842_130853_155471_179534,intersection_3


# 地図

In [82]:
import folium

In [83]:
m = folium.Map(
    location=[df5.lat.mean(), df5.lon.mean()],
    zoom_start=12,
    tiles="https://cyberjapandata.gsi.go.jp/xyz/pale/{z}/{x}/{y}.png",
    attr='&copy; <a href="https://maps.gsi.go.jp/development/ichiran.html">国土地理院</a>',
)

In [84]:
for i, r in df5.iterrows():

    folium.Marker(
        location=[r.lat, r.lon],
        icon=folium.Icon(color="red")
    ).add_to(m)

In [85]:
m